### REQUIREMENTS OF DASHBOARD
1.  visuals of 'ON TIME', 'IN FULL' AND 'OTIF %' (ON TIME IN FULL) FOR ALL ORDERS. (SHOW THIS IN RESPECT TO TARGETS TO METRICS.)
2.  visuals should be split by CITIES and CUSTOMERS.
3.  visuals of 'LINE FILL RATE' and 'VOLUME FULL RATE'
4.  They want to understand the customer's service level visually over all metrics with respect to target.
5.  Customers and all metrics in a matrix visual:
    - Apply conditional formatting on values based on their gap with respect to the target.
6. Show the OTIF performance VS target over months. (It should be drillable for weeks and days)
7. When selecting metrics, the graph should change.
8. show LIFR (LINE FILL RATE) and VOFR (VOLUME FULL RATE) of products with a SPARKLINE next to it.

In [1]:
import pandas as pd

In [2]:
dim_customers = pd.read_csv('dim_customers.csv')
dim_date = pd.read_csv('dim_date.csv')
dim_products = pd.read_csv('dim_products.csv')
dim_target_orders = pd.read_csv('dim_targets_orders.csv')
fact_order_lines = pd.read_csv('fact_order_lines.csv')
fact_orders_aggregate = pd.read_csv('fact_orders_aggregate.csv')

In [23]:
all_data = [dim_customers, dim_date, dim_products, dim_target_orders,
        fact_order_lines, fact_orders_aggregate]
all_data_names = ['dim_customers', 'dim_date', 'dim_products', 'dim_target_orders',
        'fact_order_lines', 'fact_orders_aggregate']

In [27]:
for data_name, data in zip(all_data_names, all_data):
    print(data_name)
    print(f'{"+"*30}')
    print(data.head())
    print(data.info())
    print(f"{'-'*100}")

dim_customers
++++++++++++++++++++++++++++++
   customer_id      customer_name       city
0       789201          Rel Fresh      Surat
1       789202          Rel Fresh  Ahmedabad
2       789203          Rel Fresh   Vadodara
3       789301  Expression Stores      Surat
4       789303  Expression Stores   Vadodara
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    35 non-null     int64 
 1   customer_name  35 non-null     object
 2   city           35 non-null     object
dtypes: int64(1), object(2)
memory usage: 968.0+ bytes
None
----------------------------------------------------------------------------------------------------
dim_date
++++++++++++++++++++++++++++++
        date     mmm_yy week_no
0  01-Apr-22  01-Apr-22    W 14
1  03-Apr-22  01-Apr-22    W 15
2  04-Apr-22  01-Apr-22    W 15
3  05-Apr-22  01-Apr-22    W 15
4  06-A

In [67]:
# ontime_target
# No of orders delivered On Time / Total Number of Orders
onTime_orders = fact_orders_aggregate[fact_orders_aggregate['on_time'] == 1]
onTime_orders = onTime_orders.groupby('customer_id')['on_time'].count()\
    .reset_index().sort_values('customer_id')
# onTime_orders
total_orders = fact_orders_aggregate.groupby('customer_id')['on_time'].count()\
    .reset_index().sort_values('customer_id')
print(onTime_orders.head())
total_orders.head()

   customer_id  on_time
0       789101      596
1       789102      581
2       789103      611
3       789121      361
4       789122      349


,customer_id,on_time
0,789101,821
1,789102,832
2,789103,815
3,789121,1219
4,789122,1218


In [68]:
orders = onTime_orders.merge(total_orders, on='customer_id', suffixes=['_actual', '_total'])
orders.head()

,customer_id,on_time_actual,on_time_total
0,789101,596,821
1,789102,581,832
2,789103,611,815
3,789121,361,1219
4,789122,349,1218


In [69]:
orders['ontime_actual%'] = orders['on_time_actual']/orders['on_time_total'] *100
orders.head()

,customer_id,on_time_actual,on_time_total,ontime_actual%
0,789101,596,821,72.594397
1,789102,581,832,69.831731
2,789103,611,815,74.969325
3,789121,361,1219,29.614438
4,789122,349,1218,28.653530


In [70]:
# Drop columns not needed
orders = orders.drop(axis=1, columns=['on_time_actual', 'on_time_total'])
orders.head()

,customer_id,ontime_actual%
0,789101,72.594397
1,789102,69.831731
2,789103,74.969325
3,789121,29.614438
4,789122,28.653530


In [71]:
# I merged to ensure it captures only customers in both tables.
# Just incase there is a customer only available in both tables.
metrics_vs_targets = orders.merge(dim_target_orders, on='customer_id')
metrics_vs_targets.head()

,customer_id,ontime_actual%,ontime_target%,infull_target%,otif_target%
0,789101,72.594397,86,80,69
1,789102,69.831731,90,81,73
2,789103,74.969325,92,67,62
3,789121,29.614438,78,77,60
4,789122,28.653530,76,65,49


In [72]:
# infull_target
# No of orders delivered In Full / Total Number of Orders
inFull_orders = fact_orders_aggregate[fact_orders_aggregate['in_full'] == 1]
inFull_orders = inFull_orders.groupby('customer_id')['in_full'].count()\
    .reset_index().sort_values('customer_id')
# onTime_orders
total_orders = fact_orders_aggregate.groupby('customer_id')['in_full'].count()\
    .reset_index().sort_values('customer_id')
print(inFull_orders.head())
total_orders.head()

   customer_id  in_full
0       789101      479
1       789102      485
2       789103      146
3       789121      819
4       789122      271


,customer_id,in_full
0,789101,821
1,789102,832
2,789103,815
3,789121,1219
4,789122,1218


In [73]:
orders = inFull_orders.merge(total_orders, on='customer_id', suffixes=['_actual', '_total'])
orders.head()

,customer_id,in_full_actual,in_full_total
0,789101,479,821
1,789102,485,832
2,789103,146,815
3,789121,819,1219
4,789122,271,1218


In [74]:
orders['infull_actual%'] = (orders['in_full_actual']/orders['in_full_total']) *100
orders.head()

,customer_id,in_full_actual,in_full_total,infull_actual%
0,789101,479,821,58.343484
1,789102,485,832,58.293269
2,789103,146,815,17.914110
3,789121,819,1219,67.186218
4,789122,271,1218,22.249589


In [75]:
# Drop columns not needed
orders = orders.drop(axis=1, columns=['in_full_actual', 'in_full_total'])
orders.head()

,customer_id,infull_actual%
0,789101,58.343484
1,789102,58.293269
2,789103,17.914110
3,789121,67.186218
4,789122,22.249589


In [76]:
metrics_vs_targets = metrics_vs_targets.merge(orders, on='customer_id')
metrics_vs_targets.head()

,customer_id,ontime_actual%,ontime_target%,infull_target%,otif_target%,infull_actual%
0,789101,72.594397,86,80,69,58.343484
1,789102,69.831731,90,81,73,58.293269
2,789103,74.969325,92,67,62,17.914110
3,789121,29.614438,78,77,60,67.186218
4,789122,28.653530,76,65,49,22.249589


In [78]:
# infull_target
# No of orders delivered In Full / Total Number of Orders
otif_orders = fact_orders_aggregate[fact_orders_aggregate['otif'] == 1]
otif_orders = otif_orders.groupby('customer_id')['otif'].count()\
    .reset_index().sort_values('customer_id')
# onTime_orders
total_orders = fact_orders_aggregate.groupby('customer_id')['otif'].count()\
    .reset_index().sort_values('customer_id')
print(otif_orders.head())
total_orders.head()

   customer_id  otif
0       789101   313
1       789102   299
2       789103    86
3       789121   248
4       789122    87


,customer_id,otif
0,789101,821
1,789102,832
2,789103,815
3,789121,1219
4,789122,1218


In [83]:
orders = otif_orders.merge(total_orders, on='customer_id', suffixes=['_actual', '_total'])
orders.head()

,customer_id,otif_actual,otif_total
0,789101,313,821
1,789102,299,832
2,789103,86,815
3,789121,248,1219
4,789122,87,1218


In [84]:
orders['otif_actual%'] = (orders['otif_actual']/orders['otif_total']) *100
orders.head()

,customer_id,otif_actual,otif_total,otif_actual%
0,789101,313,821,38.124239
1,789102,299,832,35.937500
2,789103,86,815,10.552147
3,789121,248,1219,20.344545
4,789122,87,1218,7.142857


In [85]:
# Drop columns not needed
orders = orders.drop(axis=1, columns=['otif_actual', 'otif_total'])
orders.head()

,customer_id,otif_actual%
0,789101,38.124239
1,789102,35.937500
2,789103,10.552147
3,789121,20.344545
4,789122,7.142857


In [86]:
metrics_vs_targets = metrics_vs_targets.merge(orders, on='customer_id')
metrics_vs_targets.head()

,customer_id,ontime_actual%,ontime_target%,infull_target%,otif_target%,infull_actual%,otif_actual%
0,789101,72.594397,86,80,69,58.343484,38.124239
1,789102,69.831731,90,81,73,58.293269,35.937500
2,789103,74.969325,92,67,62,17.914110,10.552147
3,789121,29.614438,78,77,60,67.186218,20.344545
4,789122,28.653530,76,65,49,22.249589,7.142857


In [88]:
metrics_vs_targets[['ontime_actual%', 'ontime_target%']].head()

,ontime_actual%,ontime_target%
0,72.594397,86
1,69.831731,90
2,74.969325,92
3,29.614438,78
4,28.653530,76


In [89]:
metrics_vs_targets.to_csv('metrics_vs_targets.csv')